In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression

df = pd.read_excel('ANZ synthesised transaction dataset.xlsx')

# 7717 comprises of SALES-POS and POS Transaction Types
# 885 for bpay_biller_code comprises of 883 for PAY/SALARY, 1 for POS and 1 for SALES-POS
# 883 for merchant_code are all PAY/SALARY

In [2]:
# Extract transactions that include salary payments for customers
salary = df[df['txn_description'] == 'PAY/SALARY']
salary = salary.drop(['card_present_flag', 'merchant_id', 'merchant_suburb', 'merchant_state', 'merchant_long_lat'], axis=1)
salary = pd.DataFrame(salary.groupby(['customer_id', 'date','amount','gender','age'])['amount'].unique())
salary = salary.drop('amount', axis=1).reset_index()

In [3]:
# Add missing data for the 16/8/18

data = [{'customer_id': 'CUS-1233833708', 'date': '2018-08-16', 'amount': 596.33, 'gender': 'M', 'age': 44},
        {'customer_id': 'CUS-1646183815', 'date': '2018-08-16', 'amount': 782.42, 'gender': 'M', 'age': 78},
        {'customer_id': 'CUS-2283904812', 'date': '2018-08-16', 'amount': 2218.73, 'gender': 'M', 'age': 22},
        {'customer_id': 'CUS-2599279756', 'date': '2018-08-16', 'amount': 1630.36, 'gender': 'F', 'age': 43},
        {'customer_id': 'CUS-2630892467', 'date': '2018-08-16', 'amount': 920.71, 'gender': 'F', 'age': 26},
        {'customer_id': 'CUS-3201519139', 'date': '2018-08-16', 'amount': 916.75, 'gender': 'M', 'age': 37},
        {'customer_id': 'CUS-3462882033', 'date': '2018-08-16', 'amount': 3195.01, 'gender': 'M', 'age': 22},
        {'customer_id': 'CUS-72755508', 'date': '2018-08-16', 'amount': 725.32, 'gender': 'F', 'age': 35},
        {'customer_id': 'CUS-860700529', 'date': '2018-08-16', 'amount': 1808.62, 'gender': 'M', 'age': 30},
        {'customer_id': 'CUS-880898248', 'date': '2018-08-16', 'amount': 1433.98, 'gender': 'M', 'age': 26}]

missing = pd.DataFrame(data)
missing['date'] = pd.to_datetime(missing['date'])
salary  = pd.concat([salary, missing])
salary = salary.sort_values(['customer_id', 'date'])

In [4]:
# Calculate payment periods in days
salary['pay_period'] = salary.groupby('customer_id')['date'].apply(lambda x: x.shift(-1) - x)
salary['pay_period'] = salary['pay_period'].dt.days
salary = salary.dropna()

In [5]:
# Calculate frequency of payments per year
salary['pay_freq'] = 52
salary.loc[salary['pay_period'] == 14.0, 'pay_freq'] = 26
salary.loc[salary['pay_period'] > 28.0, 'pay_freq'] = 12
salary.loc[salary['pay_period'] > 58.0, 'pay_freq'] = 6
salary = salary.drop(['date', 'pay_period'], axis=1)
salary = salary.drop_duplicates()

In [6]:
# Determine Annual Salary
salary['annual_salary'] = salary['amount'] * salary['pay_freq']

In [7]:
salary.reset_index(drop=True, inplace=True)
salary

,customer_id,amount,gender,age,pay_freq,annual_salary
0,CUS-1005756958,970.47,F,53,52,50464.44
1,CUS-1117979751,3578.65,M,21,26,93044.90
2,CUS-1140341822,1916.51,M,28,26,49829.26
3,CUS-1147642491,1711.39,F,34,52,88992.28
4,CUS-1196156254,3903.73,F,34,26,101496.98
...,...,...,...,...,...,...
95,CUS-72755508,725.32,F,35,52,37716.64
96,CUS-809013380,1037.07,F,21,52,53927.64
97,CUS-860700529,1808.62,M,30,26,47024.12
98,CUS-880898248,1433.98,M,26,26,37283.48


In [8]:
# Export csv file containing the customer id and annual salary
annual_salary = salary.drop(['amount', 'gender', 'age', 'pay_freq'], axis=1)
export_csv = annual_salary.to_csv (r'annual_salary.csv', header=True, index = False)

In [9]:
# Export salary as csv file for further analysis in R
export_csv = salary.to_csv (r'customer_id.csv', header=True, index = False)

# Merge annual_salary with df

In [10]:
# Drop columns that already exist in df
salary = salary.drop(['amount','gender','age'], axis=1)

In [11]:
# Merge annual salary with full dataset
df = pd.merge(df, salary, on='customer_id')

In [12]:
# Export full dataframe with annual salary column for further analysis in R
export_csv = df.to_csv (r'full_plus_as.csv', header=True, index = False)

Keep as may need later

In [ ]:
# df[['long','lat']] = df.long_lat .str.split(" ",expand=True,)

In [ ]:
# df['long'] = df['long'].astype(float)
# df['lat'] = df['lat'].astype(float)